In [ ]:
from nltk import sent_tokenize
import numpy as np
from bert_ner_utils import BERT_NER

In [ ]:
# Importing models
import spacy
nlp = spacy.load("en_core_web_sm")

ner_model = BERT_NER('/home/arjun/BERT_Similarity_experiments/models/bert-ner/',cased=False)
ner_model_u = BERT_NER('/home/arjun/BERT_Similarity_experiments/models/bert-ner-uncased-2/',cased=False,labels=["O","E"])
# ner_model_c = BERT_NER('/home/arjun/BERT_Similarity_experiments/models/bert-ner-cased/',cased=True)
ner_model_c = BERT_NER('/home/arjun/BERT_Similarity_experiments/models/bert-ner-cased-2/',cased=True,labels=["O","E"])

In [ ]:
test_segment = "'gotham city needs a #Wash', said batman in his low, grumbly voice as he waved his wand to cast the spell aguamenti on the city."
model = ner_model_u

ents,sc,nents,nsc = model.get_entities(test_segment,get_non_entities=True)
print(">",test_segment,end="\n\n")
print("Entities Detected:",ner_model.wordize(ents, capitalize=True),end="\n\n")
print("Non Entity tokens, sorted by score:",end="\n\n")

for ne,ns in sorted(zip(nents,nsc),key=lambda x: x[1]):
    print(ne,ns,sep="\t")


## Comparing models based on segment

In [ ]:
segment = "most of you. How many of you actually used one? No, that's that's good. I gave this talk a similar talk. Clashing there was, too. So now we're up to about 10 or 15. How many of you actually put something into production with one? We're down about four. How many of you actually put it into production successfully? We're down to a couple. So that's sort of what the whole premise of this talk is to kind of walk through the process of what it actually takes to make a raft database implementation. Successful. Ah, so little bit about myself before we get started. So, uh, I've been doing this. I've been doing software development for about 20 years. Every time I say that, it starts to make me feel older and older a little more than 20 years now, But I don't want to admit how long. Ah, and I've done everything. I've been a little bit of everything. I started out. My actual first job was to build embedded, see programs on top of dos on. Other than that, everything they're through building out high through building out high performance websites. Most recently, I've been focused on big data problems specifically around building. Ah, high performance, low latent See data platforms. A lot of times, at least part of the Zeta platform has been leveraging crap leveraging a draft database. Currently, I am, ah, solutions, architect. Data s*x in the global graft practice. If you're not familiar with the data stacks is we are the Apache Ca*sandra people. Ah, we have our own distribution of Ca*sandra that integrates search a**lytics and then Graff on top of it, which is, uh, why I'm part of the global graph practice there. Most of the co author of an upcoming book by Manny called Graph Databases in Action On And if you are, if you want my that's my Twitter handle up in the corner. If you tweet out during this this presentation I have five free copies like I'll give away at the end. Doesn't have to be nice, but I would prefer it be nice. "
print(segment,end="\n\n")
entities=[]
entities_c = []
entities_u = []
non_entities_u=[]
conf = []
conf_c = []
conf_u=[]
non_conf_u=[]
for text in sent_tokenize(segment):
#     print("Entities Uncased Mul")
    ent, con, ne,nc = ner_model.get_entities(text,get_non_entities=True)
#     print("Entities Cased Bin")
    ent_c, con_c,ne_c, nc_c = ner_model_c.get_entities(text,get_non_entities=True)
#     print("Entities Uncased Bin")
    ent_u, con_u,ne_u,nc_u = ner_model_u.get_entities(text,get_non_entities=True)
#     print()
    if len(ent)==0 and len(ent_c)==0 and len(ent_u)==0:
        continue
    entities.extend(ent)
    entities_c.extend(ent_c)
    entities_u.extend(ent_u)
    non_entities_u.extend(ne_u)
    non_conf_u.extend(nc_u)
    conf.extend(con)
    conf_c.extend(con_c)
    conf_u.extend(con_u)

print("UNCASED BERT-NER",end="\n\n")
[print(e,c) for e,c in zip(entities,conf)]
print(len(entities),len(conf))
entities_words = ner_model.wordize(entities)
print(entities_words,end="\n\n")

print("-"*100)
print("UNCASED BERT-NER-BINARY",end="\n\n")
[print(e,c) for e,c in zip(entities_u,conf_u)]
print("#"*100)
[print(e,c) for e,c in sorted(zip(non_entities_u,non_conf_u),key=lambda x: x[1])]
entities_words_u = ner_model_u.wordize(entities_u)
print(entities_words_u,end="\n\n")

print("-"*100)
print("CASED BERT-NER-BINARY",end="\n\n")
# [print(e,c) for e,c in zip(entities_c,conf_c)]
entities_words_c = ner_model_c.wordize(entities_c)
print(entities_words_c,end="\n\n")

print("-"*100)
print("SPACY")
doc=nlp(segment)
entities_spacy=[]
for ent in doc.ents:
    if ent.label_ not in ["CARDINAL","ORDINAL"]:
        entities_spacy.append(ent.text)
print(", ".join(list(set(entities_spacy))))

# Validation Tests

## Validation on entity-sentences set

In [ ]:
import pickle
with open("/home/venkat/knowledge_graphs/entity_graph_builder/graph_dumps/ppn_sentences.pkl","rb") as f:
    sent_dict = pickle.load(f)

In [ ]:
ctr = 0
num_of_entities=10
for gold_entity in sent_dict:
    entities = []
    entities_c = []
    entities_u = []
    entities_spacy_u = []
    entities_spacy_c = []
    # Selecting entities based on starting letter
#     if gold_entity!="Fix":
#         continue
    gold_sentences = sent_dict[gold_entity][:500]
    gold_entity = gold_entity.replace("the ","")
    if len(gold_sentences)<4:
        continue
    
    for text in gold_sentences:
        ent, con= ner_model.get_entities(text)
        ent_c, con_c= ner_model_c.get_entities(text)
        ent_u, con_u= ner_model_u.get_entities(text)
        
        doc_c = nlp(text)
        entities_spacy_c.extend(list(set([x for e in doc_c.ents for x in e.text.upper().split() ])))
        doc_u = nlp(text.lower())
        entities_spacy_u.extend(list(set([x for e in doc_u.ents for x in e.text.upper().split() ])))
        
        if len(ent)==0 and len(ent_c)==0 and len(ent_u)==0:
            continue
        entities.extend(list(set(ner_model.wordize(ent))))
        entities_c.extend(list(set(ner_model_c.wordize(ent_c))))
        entities_u.extend(list(set(ner_model_u.wordize(ent_u))))
    
    # Consider n-gram entities. Take mean score of all entity detections.
    gold_entity_list = gold_entity.upper().split()
    counts_1 = np.mean([entities.count(gold_ent) for gold_ent in gold_entity_list])
    counts_2 = np.mean([entities_c.count(gold_ent) for gold_ent in gold_entity_list])
    counts_3 = np.mean([entities_u.count(gold_ent) for gold_ent in gold_entity_list])
    counts_ner_spacy_cased = entities_spacy_c.count(gold_entity.upper())
    counts_ner_spacy_uncased = entities_spacy_u.count(gold_entity.upper())
    
    print("#"*100,"\n",gold_entity,"| #Sentences:",len(gold_sentences),"\n")
    print("BERT NER UNCASED MUL: ", counts_1/len(gold_sentences))
    print("BERT NER CASED BIN: ", counts_2/len(gold_sentences))
    print("BERT NER UNCASED BIN: ", counts_3/len(gold_sentences))
    print("SPACY NER CASED: ", counts_ner_spacy_cased/len(gold_sentences))
    print("SPACY NER UNCASED: ", counts_ner_spacy_uncased/len(gold_sentences))
    
    ctr+=1
    if ctr==num_of_entities:
        break

In [ ]:
gold_sentences

In [ ]:
model = ner_model_u
for text in sent_dict['Atlassian'][:10]:
    print(text)
    ents = model.get_entities(text,get_non_entities=True)[0]
    print(model.wordize(ents,capitalize=True))
    print()

## Validation on Meeting Segments

In [ ]:
import json
with open("/home/arjun/BERT_Similarity_experiments/data/entity_val_pl1.json","r") as f:
    entity_val_set = json.loads(f.read().replace("][",","))

In [ ]:
for seg_chunk in entity_val_set:
    gold_entity_list = list(seg_chunk['entities'].keys())
    entities = []
    entities_c = []
    entities_u = []
    entities_spacy_u = []
    entities_spacy_c = []
    conf=[]
    conf_c=[]
    conf_u=[]
    print(seg_chunk['segments'],end="\n\n")
    gold_sentences = sent_tokenize(seg_chunk['segments'])
    for text in gold_sentences:
#         text=text.lower() # ABLATION STUDY FOR BERT CASED MODEL
        ent, con= ner_model.get_entities(text)
        ent_c, con_c= ner_model_c.get_entities(text)
        ent_u, con_u= ner_model_u.get_entities(text)
        entities.extend(ent)
        entities_c.extend(ent_c)
        entities_u.extend(ent_u)
        conf.extend(con)
        conf_c.extend(con_c)
        conf_u.extend(con_u)
        
        doc_c = nlp(text)
        entities_spacy_c.extend(list(set([e.text for e in doc_c.ents])))
        doc_u = nlp(text.lower())
        entities_spacy_u.extend(list(set([e.text for e in doc_u.ents])))
    entities = ner_model.wordize(entities, capitalize=True)
    entities_c = ner_model_c.wordize(entities_c, capitalize=True)
    entities_u = ner_model_u.wordize(entities_u, capitalize=True)
    
    print("\n",gold_entity_list,"No. of Sentences:",len(gold_sentences),"\n")
    print("BERT NER UNCASED MUL: ",entities,conf,sep="\n",end="\n\n")
    print("BERT NER CASED BIN: ",entities_c,conf_c,sep="\n",end="\n\n")
    print("BERT NER UNCASED BIN: ",entities_u,conf_u,sep="\n",end="\n\n")
    print("SPACY NER CASED",set(entities_spacy_c))
    print("SPACY NER UNCASED",set(entities_spacy_u))
    print("#"*100)